In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
#import seaborn as sns

One tailed t-test - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

In [120]:
with open('files_for_lab/machine.txt', 'rb') as f:
  contents = f.read()
contents = contents.decode('utf-16', 'ignore') 
contents=contents.split('\r\n')
contents = pd.DataFrame(contents)
contents[['new','old']] = contents[0].str.split('\t', expand = True)
contents=contents[['new','old']]
contents=contents.loc[1:10]
contents['new'] =  pd.to_numeric(contents['new'], errors='coerce')
contents['old'] =  pd.to_numeric(contents['old'], errors='coerce')
contents

,new,old
1,42.1,42.7
2,41.0,43.6
3,41.3,43.8
4,41.8,43.3
5,42.4,42.5
6,42.8,43.5
7,43.2,43.1
8,42.3,41.7
9,41.8,44.0
10,42.7,44.1


In [132]:
new_mean = contents['new'].mean()
old_mean = contents['old'].mean()

sample_old_mean, sample_new_mean = contents['old'].mean(), contents['new'].mean()
sample_old_std, sample_new_std = contents['old'].std(ddof=1), contents['new'].std(ddof=1)
sp = ( contents['old'].shape[0] - 1 ) * ( sample_old_std**2 ) +  ( contents['new'].shape[0] - 1 ) * ( sample_new_std**2 )
sp /= ( contents['old'].shape[0] + contents['new'].shape[0] - 2)
r = np.sqrt( (1/contents['old'].shape[0]) + (1/contents['new'].shape[0]) )
t = ( sample_new_mean - sample_old_mean )/ (sp * r)

print("The mean of sample 'old' is {:.2f}".format(sample_old_mean))
print("The mean of sample 'new' is {:.2f}".format(sample_new_mean))
print("The standard deviation of sample 'old' is {:.2f}".format(sample_old_std))
print("The standard deviation of sample 'new' is {:.2f}".format(sample_new_std))
print("The ratio of the sample variances is {:.2f} which is bigger than 0.5 and smaller than 2".format(sample_old_std/sample_new_std))
print("The t statistic is: {:.2f}".format(t))

The mean of sample 'old' is 43.23
The mean of sample 'new' is 42.14
The standard deviation of sample 'old' is 0.75
The standard deviation of sample 'new' is 0.68
The ratio of the sample variances is 1.10 which is bigger than 0.5 and smaller than 2
The t statistic is: -4.74


In [134]:
#one-tailed test (the new machine is faster than the old machine)
st.t.ppf(1-0.05,df = contents['old'].shape[0]+contents['new'].shape[0]-2)
#t-statistics is higher than the trashhold statistic

1.7340636066175354

Matched Pairs Test - In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [135]:
data = pd.read_csv('files_for_lab/pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [145]:
sample = data.sample(30, random_state=1)
sample['difference'] = (sample['Attack']-sample['Defense'])/sample['Attack'] # scaled difference
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(-0.0028169117732530853, 0.4094811686965061)

In [146]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: -0.00
The standard deviation of our samples differences is: 0.41
Our t statistics is: -0.04


In [147]:
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703

In [ ]:
# there is no significant difference between attack and defence. T-statistics inside the acceptance region. 
# we accept the null (dif = 0, mean attack = mean defence)

Inferential statistics - ANOVA

Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of model, error terms, and total DoF
Data was collected randomly and provided to you in the table as shown: link to the image - Data

In [159]:
# the null is that the etching is the same for each power level
# the alternative is that the etching is not the same for each power level
# the significance level is 5%
# degrees of freedom between groups is 3-1 = 2
# degrees of freedom within groups is 15-3 = 12
# degrees of freedom total is 15-1 = 14

data = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
data.columns=data.columns.str.strip()
data['Power'].value_counts()

160 W    5
180 W    5
200 W    5
Name: Power, dtype: int64

In [164]:
group_df = data.groupby('Power')['Etching Rate'].agg(Mean='mean',Samples='size').reset_index()
group_df

,Power,Mean,Samples
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


In [165]:
S2t = 0
for power in data['Power'].unique():
    ng = len(data[data['Power'] == power])  
    S2t  += ( ( data[data['Power'] == power]['Etching Rate'].mean() - data['Etching Rate'].mean() ) ** 2) * ng
S2t /= ( data['Power'].nunique() - 1 )
print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


In [167]:
S2E = 0
for power in data['Power'].unique():
    for rate in data[data['Power'] == power]['Etching Rate']:
        S2E += ( rate - data[data['Power'] == power]['Etching Rate'].mean() ) ** 2
S2E /= ( len(data) - data['Power'].nunique() ) 

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [168]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [170]:
st.f_oneway(data[data['Power'] == '160 W']['Etching Rate'],
            data[data['Power'] == '180 W']['Etching Rate'],
            data[data['Power'] == '200 W']['Etching Rate']
            )

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

In [ ]:
# the F-statistic is far right to the safe interval (36 vs 3.44), we reject the null, 
# the means are not the same

In [183]:
import scipy.stats
scipy.stats.f.ppf(95/100, 2,12)

3.8852938346523933